In [2]:
## Celda 1: Instalación de dependencias
# Ejecuta esta celda primero para instalar las dependencias necesarias
!apt-get update
!apt install chromium-chromedriver
!pip install selenium beautifulsoup4 pandas

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"apt" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
## Celda 2: Importación de librerías
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [4]:
## Celda 3: Configuración y funciones principales

# ====== CONFIGURACIÓN MODIFICABLE ======
# Cambia estas variables según tus necesidades
PALABRAS_CLAVE = "guajira, renovable"  # ← Modifica aquí las palabras de búsqueda
MAX_PAGINAS = None  # ← None para todas las páginas, o un número específico (ej: 5)

# ====== FUNCIONES DEL SCRAPER ======

def configurar_driver():
    """Configura Chrome para Google Colab"""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
    return webdriver.Chrome(options=chrome_options)

def extraer_enlaces_pagina(driver):
    """Extrae enlaces de la página actual"""
    enlaces = []
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "queryly_item_row"))
        )
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        items = soup.find_all('div', class_='queryly_item_row')
        
        for item in items:
            link_element = item.find('a')
            if link_element and link_element.get('href'):
                href = link_element['href']
                url_completa = f"https://www.elheraldo.co{href}" if not href.startswith('http') else href
                
                titulo = item.find('div', class_='queryly_item_title')
                titulo_texto = titulo.get_text(strip=True) if titulo else ""
                
                descripcion = item.find('div', class_='queryly_item_description')
                desc_texto = descripcion.get_text(strip=True) if descripcion else ""
                
                fecha = item.find('div', style=lambda value: value and 'color:#555' in value)
                fecha_texto = fecha.get_text(strip=True) if fecha else ""
                
                enlaces.append({
                    'url': url_completa,
                    'titulo': titulo_texto,
                    'descripcion': desc_texto,
                    'fecha': fecha_texto
                })
    except Exception as e:
        print(f"Error al extraer enlaces: {e}")
    return enlaces

def ir_siguiente_pagina(driver):
    """Navega a la siguiente página"""
    try:
        boton = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "next_btn"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", boton)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", boton)
        time.sleep(3)
        return True
    except:
        return False


In [5]:
## Celda 4: Ejecutar el scraping

# Preparar URL con las palabras clave
keywords_url = PALABRAS_CLAVE.replace(" ", "%20").replace(",", "%2C")
url = f"https://www.elheraldo.co/buscador/?query={keywords_url}"

print(f"🔍 Buscando: '{PALABRAS_CLAVE}'")
print(f"📌 URL: {url}\n")

# Inicializar driver y variables
driver = configurar_driver()
todos_los_enlaces = []
pagina = 1

try:
    # Cargar primera página
    driver.get(url)
    time.sleep(3)
    
    while True:
        print(f"📄 Página {pagina}...")
        
        # Extraer enlaces
        enlaces = extraer_enlaces_pagina(driver)
        if enlaces:
            todos_los_enlaces.extend(enlaces)
            print(f"   ✅ {len(enlaces)} enlaces encontrados (Total: {len(todos_los_enlaces)})")
        
        # Verificar límite de páginas
        if MAX_PAGINAS and pagina >= MAX_PAGINAS:
            print(f"\n🛑 Límite de {MAX_PAGINAS} páginas alcanzado")
            break
        
        # Ir a siguiente página
        if not ir_siguiente_pagina(driver):
            print(f"\n✅ Scraping completado - Última página alcanzada")
            break
        
        pagina += 1

finally:
    driver.quit()
    print(f"\n📊 Total de enlaces extraídos: {len(todos_los_enlaces)}")


🔍 Buscando: 'guajira, renovable'
📌 URL: https://www.elheraldo.co/buscador/?query=guajira%2C%20renovable

📄 Página 1...
   ✅ 20 enlaces encontrados (Total: 20)
📄 Página 2...
   ✅ 20 enlaces encontrados (Total: 40)
📄 Página 3...
   ✅ 20 enlaces encontrados (Total: 60)
📄 Página 4...
   ✅ 20 enlaces encontrados (Total: 80)
📄 Página 5...
   ✅ 20 enlaces encontrados (Total: 100)
📄 Página 6...
   ✅ 20 enlaces encontrados (Total: 120)
📄 Página 7...
   ✅ 20 enlaces encontrados (Total: 140)
📄 Página 8...
   ✅ 20 enlaces encontrados (Total: 160)
📄 Página 9...
   ✅ 18 enlaces encontrados (Total: 178)

✅ Scraping completado - Última página alcanzada

📊 Total de enlaces extraídos: 178


In [6]:
## Celda 5: Ver resultados
# Mostrar primeros 10 resultados
print("🔍 PRIMEROS 10 RESULTADOS:\n")
for i, enlace in enumerate(todos_los_enlaces[:10], 1):
    print(f"{i}. {enlace['titulo'][:70]}...")
    print(f"   📅 {enlace['fecha']}")
    print(f"   🔗 {enlace['url']}\n")

🔍 PRIMEROS 10 RESULTADOS:

1. Retraso en proyectos renovables pone en riesgo estabilidad eléctrica...
   📅 sáb, 15 nov 2025
   🔗 https://www.elheraldo.co/atlantico/2025/11/16/al-gobierno-lo-cogio-la-tarde-expertos-alertan-retrasos-en-proyectos-de-energia-claves-para-el-pais/

2. Aprendiz del Sena La Guajira representará a Colombia en WorldSkills Sh...
   📅 jue, 9 oct 2025
   🔗 https://www.elheraldo.co/la-guajira/2025/10/09/aprendiz-del-sena-la-guajira-representara-a-colombia-en-worldskills-shanghai-2026/

3. “Licenciamientos de proyectos renovables no convencionales en La Guaji...
   📅 jue, 19 jun 2025
   🔗 https://www.elheraldo.co/la-guajira/2025/06/19/licenciamientos-de-proyectos-renovables-no-convencionales-en-la-guajira-estan-al-dia-anla/

4. Isagen fortalece su apuesta por las energías renovables  en el norte d...
   📅 sáb, 25 oct 2025
   🔗 https://www.elheraldo.co/atlantico/2025/10/26/isagen-fortalece-su-apuesta-por-las-energias-renovables-en-el-norte-de-colombia/

5. Anuncian in

In [7]:
## Celda 6: Crear lista simple de URLs
# Extraer solo las URLs en una lista simple
urls_listas_para_scraping = [enlace['url'] for enlace in todos_los_enlaces]

print(f"📋 Lista de {len(urls_listas_para_scraping)} URLs listas para siguiente scraping:\n")

# Mostrar primeras 5 URLs
for i, url in enumerate(urls_listas_para_scraping[:5], 1):
    print(f"{i}. {url}")

print("\n💡 Usa la variable 'urls_listas_para_scraping' para tu siguiente proceso")

📋 Lista de 178 URLs listas para siguiente scraping:

1. https://www.elheraldo.co/atlantico/2025/11/16/al-gobierno-lo-cogio-la-tarde-expertos-alertan-retrasos-en-proyectos-de-energia-claves-para-el-pais/
2. https://www.elheraldo.co/la-guajira/2025/10/09/aprendiz-del-sena-la-guajira-representara-a-colombia-en-worldskills-shanghai-2026/
3. https://www.elheraldo.co/la-guajira/2025/06/19/licenciamientos-de-proyectos-renovables-no-convencionales-en-la-guajira-estan-al-dia-anla/
4. https://www.elheraldo.co/atlantico/2025/10/26/isagen-fortalece-su-apuesta-por-las-energias-renovables-en-el-norte-de-colombia/
5. https://www.elheraldo.co/la-guajira/2025/06/19/anuncian-incentivo-de-mas-de-96-mil-millones-para-impulsar-el-desarrollo-sostenible-en-la-guajira/

💡 Usa la variable 'urls_listas_para_scraping' para tu siguiente proceso


In [10]:
## Celda 7: Guardar resultados

# Guardar en CSV con toda la información
df = pd.DataFrame(todos_los_enlaces)
df.to_csv(f'heraldo_resultados_{PALABRAS_CLAVE}.csv', index=False, encoding='utf-8')
print(f"💾 Resultados completos guardados en: heraldo_resultados_{PALABRAS_CLAVE}.csv")

# Guardar solo URLs en archivo de texto
with open(f'heraldo_urls_{PALABRAS_CLAVE}.txt', 'w', encoding='utf-8') as f:
    for url in urls_listas_para_scraping:
        f.write(url + '\n')
print(f"💾 Lista de URLs guardada en: heraldo_urls_{PALABRAS_CLAVE}.txt")



💾 Resultados completos guardados en: heraldo_resultados_guajira, renovable.csv
💾 Lista de URLs guardada en: heraldo_urls_guajira, renovable.txt


In [9]:

print("\n🔍 PALABRAS CLAVE EN TÍTULOS:")
for palabra in PALABRAS_CLAVE.split(','):
    palabra = palabra.strip()
    count = df['titulo'].str.contains(palabra, case=False, na=False).sum()
    print(f"  '{palabra}': {count} menciones")


🔍 PALABRAS CLAVE EN TÍTULOS:
  'guajira': 31 menciones
  'renovable': 16 menciones
